In [1]:
import setup
setup.init_django()

In [2]:
from market import services as market_services

In [3]:
def get_stock_indicators(ticker = "AAPL", days=30):
    queryset = market_services.get_daily_stock_quotes_queryset(ticker, days=days)
    if queryset.count() == 0:
        raise Exception(f"Data for {ticker} not found")
    averages = market_services.get_daily_moving_averages(ticker, days=days, queryset=queryset)
    price_target = market_services.get_price_target(ticker, days=days, queryset=queryset)
    volume_trend = market_services.get_volume_trend(ticker, days=days, queryset=queryset)
    rsi_data = market_services.calculate_rsi(ticker, days=days, period=14)
    signals = []
    if averages.get('ma_5') > averages.get('ma_20'):
        signals.append(1)
    else:
        signals.append(-1)
    if price_target.get('current_price') < price_target.get('conservative_target'):
        signals.append(1)
    else:
        signals.append(-1)
    if volume_trend.get("volume_change_percent") > 20:
        signals.append(1)
    elif volume_trend.get("volume_change_percent") < -20:
        signals.append(-1)
    else:
        signals.append(0)
    rsi = rsi_data.get('rsi')
    if rsi > 70:
        signals.append(-1)  # Overbought
    elif rsi < 30:
        signals.append(1) # Oversold
    else:
        signals.append(0)
    return {
        "score": sum(signals),
        "ticker": ticker,
        "indicators": {
            **averages,
            **price_target,
            **volume_trend,
            **rsi_data,
        }
        
    }

In [4]:
results = get_stock_indicators(ticker='AAPL')
score = results.get("score")
results

{'score': 1,
 'ticker': 'AAPL',
 'indicators': {'ma_5': 225.55,
  'ma_20': 227.6489,
  'current_price': 227.5099,
  'conservative_target': 233.2246,
  'aggressive_target': 236.7552,
  'average_price': 228.562,
  'avg_volume': 8804.291666666666,
  'latest_volume': 103886,
  'volume_change_percent': 1079.947279499108,
  'rsi': 42.676,
  'avg_gain': 0.8386,
  'avg_loss': 1.1265,
  'period': 14,
  'days': 30}}

In [7]:
import json
from decouple import config

In [8]:
results_as_json = json.dumps(results)
results_as_json

'{"score": 1, "ticker": "AAPL", "indicators": {"ma_5": 225.55, "ma_20": 227.6489, "current_price": 227.5099, "conservative_target": 233.2246, "aggressive_target": 236.7552, "average_price": 228.562, "avg_volume": 8804.291666666666, "latest_volume": 103886, "volume_change_percent": 1079.947279499108, "rsi": 42.676, "avg_gain": 0.8386, "avg_loss": 1.1265, "period": 14, "days": 30}}'

In [9]:
OPENAI_API_KEY=config("OPENAI_API_KEY", default=None)

In [10]:
assert OPENAI_API_KEY is not None

In [11]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [20]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recommendation",
            "schema": {
                "type": "object",
                "properties": {
                    "buy": {
                        "description": "Recommend to buy stock",
                        "type": "boolean"
                    },
                    "sell": {
                        "description": "Recommend to sell stock",
                        "type": "boolean"
                    },
                    "hold": {
                        "description": "Recommend to hold stock",
                        "type": "boolean"
                    },
                    "explanation": {
                        "description": "Explanation of reasoning in 1 or 2 sentences",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

In [21]:
result = json.loads(response.choices[0].message.content)
result

{'buy': False,
 'sell': False,
 'hold': True,
 'explanation': 'The stock price of AAPL is currently below its moving averages and the RSI indicates it is close to oversold conditions. Although there is potential for a price increase towards the conservative and aggressive targets, the overall momentum suggests holding rather than buying or selling at this time.'}

In [15]:
result.get('hold') is True

True